In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mat
import seaborn as sns
sns.set(style="whitegrid", font_scale=1.1)

df_species = pd.read_csv("data/processed/species.csv")
df_sites = pd.read_csv("data/processed/sites.csv")
df_observations = pd.read_csv("data/processed/observations.csv")

In [ ]:
# Indicator 1: Species and individual richness per habitat

# Extract the point number (P<number>) from Site2 in df_sites
df_sites["Point"] = df_sites["Site2"].str.extract(r"P(\d+)").astype(int)

# Match observations to sites by Transect and Point number
df_observations = df_observations.rename(columns={
    "Nom transect": "Transect",
    "N° point": "Point"
})

# Merge observations with site info to attach habitat type
df_merged = pd.merge(
    df_observations,
    df_sites[["Transect", "Point", "Type"]],
    on=["Transect", "Point"],
    how="left"
)

# Group by habitat
grouped = df_merged.groupby("Type")

# Species richness = number of distinct species per habitat
species_richness = grouped["clean_espece"].nunique().sort_values(ascending=False)

# Total abundance = total number of individuals (Amount) per habitat
total_abundance = grouped["Amount"].sum().sort_values(ascending=False)

indicator1_df = pd.DataFrame({
    "Species richness": species_richness,
    "Total abundance": total_abundance
}).reset_index()

display(indicator1_df)


fig, axes = plt.subplots(2, 1, figsize=(12, 10))
order = indicator1_df.sort_values("Species richness", ascending=False)["Type"]

# Barplot for species richness
axes[0].bar(indicator1_df["Type"], indicator1_df["Species richness"], color="#4C72B0")
axes[0].set_title("Species Richness per Habitat", fontsize=14, fontweight="bold")
axes[0].set_ylabel("Number of distinct species")
axes[0].set_xticklabels(order, rotation=45, ha="right")
axes[0].bar_label(axes[0].containers[0], fontsize=9)

# Barplot for total abundance
axes[1].bar(indicator1_df["Type"], indicator1_df["Total abundance"], color="#55A868")
axes[1].set_title("Total Number of Individuals per Habitat", fontsize=14, fontweight="bold")
axes[1].set_ylabel("Total individuals observed")
axes[1].set_xticklabels(order, rotation=45, ha="right")
axes[1].bar_label(axes[1].containers[0], fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Check unmatched observations
unmatched = df_merged["Type"].isna().sum()
print(f"Unmatched observations (no habitat found): {unmatched}")

if unmatched > 0:
    # Observations with no matching site
    unmatched_rows = df_merged[df_merged["Type"].isna()][["Transect", "Point"]]

    # Unique pairs causing mismatches
    print("Unique (Transect, Point) pairs not found in df_sites:")
    display(unmatched_rows.drop_duplicates().sort_values(["Transect", "Point"]))

    # Compare which Transect values exist in observations but not in sites
    transect_obs = set(df_observations["Transect"].unique())
    transect_sites = set(df_sites["Transect"].unique())
    missing_transects = list(transect_obs - transect_sites)
    if missing_transects:
        print("Transect names present in observations but missing in df_sites:")
        for t in missing_transects:
            print("  -", t)
    else:
        print("All transect names exist in df_sites (the issue is likely with point numbers)")


In [19]:
# Indicator 2: Shannon diversity index and origin proportions per habitat

# Merge df_obs_sites with df_species to attach origin information
df_obs_full = pd.merge(
    df_obs_sites,
    df_species[["clean_name", "Origin"]],
    left_on="clean_espece",
    right_on="clean_name",
    how="left"
)

# Function to compute Shannon diversity index
def shannon_index(group):
    counts = group.groupby("clean_espece")["Amount"].sum()
    counts = counts[counts > 0].dropna()
    if counts.empty:
        return 0.0

    p = counts / counts.sum()
    p = p.astype(float).to_numpy()
    p = p[p > 0]
    if len(p) == 0:
        return 0.0

    return float(-np.sum(p * np.log(p)))

# Compute Shannon index and origin proportions per habitat
results = []
for habitat, g in df_obs_full.groupby("Type"):
    if g.empty:
        continue

    H = shannon_index(g)
    total = g["Amount"].sum()

    origin_counts = (
        g.groupby("Origin")["Amount"].sum().reindex([
            "Endémique des Petites Antilles",
            "Endémique de la Martinique",
            "Autochtone",
            "Exogène introduit par l'homme",
            "Exogène colonisateur naturel",
            "Migrateur",
            "Migrateur rare",
            "Marin"
        ], fill_value=0)
    )
    proportions = (origin_counts / total).to_dict()

    results.append({
        "Habitat": habitat,
        "Shannon_index": H,
        **proportions
    })

indicator2_df = pd.DataFrame(results).fillna(0)

# Compute Hmax = ln(S) correctly
species_richness = (
    df_obs_full.dropna(subset=["Type", "clean_espece"])
    .groupby("Type")["clean_espece"]
    .nunique()
)

hmax_df = pd.DataFrame({
    "Habitat": species_richness.index,
    "Hmax": np.log(species_richness.values)
})
indicator2_df = indicator2_df.merge(hmax_df, on="Habitat", how="left")
indicator2_df["Hmax"] = indicator2_df["Hmax"].fillna(0)

# Compute mean Shannon index
mean_shannon = indicator2_df["Shannon_index"].mean()

# Sort habitats for plotting
indicator2_df_sorted = indicator2_df.sort_values("Shannon_index", ascending=False)

# Compute grouped origin categories
indicator2_df_grouped = indicator2_df_sorted.copy()
indicator2_df_grouped["Native"] = (
    indicator2_df_grouped["Autochtone"]
    + indicator2_df_grouped["Endémique de la Martinique"]
    + indicator2_df_grouped["Endémique des Petites Antilles"]
)
indicator2_df_grouped["Introduced"] = (
    indicator2_df_grouped["Exogène introduit par l'homme"]
    + indicator2_df_grouped["Exogène colonisateur naturel"]
)
indicator2_df_grouped["Migratory/Marine"] = (
    indicator2_df_grouped["Migrateur"]
    + indicator2_df_grouped["Migrateur rare"]
    + indicator2_df_grouped["Marin"]
)

grouped_cols = ["Native", "Introduced", "Migratory/Marine"]
grouped_colors = ["#4C72B0", "#DD8452", "#55A868"]

# Visualization
fig, axes = plt.subplots(3, 1, figsize=(12, 16))
sns.set(style="whitegrid", font_scale=1.1)

# Shannon diversity per habitat
sns.barplot(
    data=indicator2_df_sorted,
    x="Habitat", y="Shannon_index", color="#4C72B0", ax=axes[0],
    label="Observed H'"
)

# Plot mean H' (horizontal)
axes[0].axhline(mean_shannon, color="green", linestyle=":", linewidth=2, label=f"Mean H' = {mean_shannon:.2f}")

# Plot theoretical Hmax (solid red line)
axes[0].plot(
    indicator2_df_sorted["Habitat"],
    indicator2_df_sorted["Hmax"].values,
    color="red", linestyle="-", marker="o", markersize=6, linewidth=2,
    alpha=0.8, label="Theoretical H'max = ln(S)"
)

# Add numeric labels inside bars for observed H'
for i, (hab, h_val) in enumerate(zip(indicator2_df_sorted["Habitat"], indicator2_df_sorted["Shannon_index"])):
    axes[0].text(
        i, h_val - 0.15, f"{h_val:.2f}", 
        ha="center", va="top", color="white", fontsize=9, fontweight="bold"
    )

# Add difference (Hmax − H') to the right of each bar
for i, (hab, h_val, hmax) in enumerate(zip(
    indicator2_df_sorted["Habitat"],
    indicator2_df_sorted["Shannon_index"],
    indicator2_df_sorted["Hmax"]
)):
    diff = hmax - h_val
    axes[0].text(
        i, hmax + 0.05, f"Δ={diff:.2f}", 
        ha="center", va="bottom", fontsize=9, color="red", fontweight="bold"
    )

axes[0].set_title("Shannon Diversity Index (H') per Habitat", fontsize=14, fontweight="bold")
axes[0].set_ylabel("H'")
axes[0].set_ylim(0, 4.5)
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(loc="lower right", frameon=True)

# Detailed origin composition per habitat
origins = [
    "Endémique des Petites Antilles", "Endémique de la Martinique",
    "Autochtone", "Exogène introduit par l'homme",
    "Exogène colonisateur naturel", "Migrateur", "Migrateur rare", "Marin"
]
origin_colors = sns.color_palette("tab10", n_colors=len(origins))

bottom = np.zeros(len(indicator2_df_sorted))
for i, origin in enumerate(origins):
    axes[1].bar(
        indicator2_df_sorted["Habitat"],
        indicator2_df_sorted[origin],
        bottom=bottom,
        label=origin,
        color=origin_colors[i]
    )
    bottom += indicator2_df_sorted[origin].values

axes[1].set_title("Species Origin Composition per Habitat", fontsize=14, fontweight="bold")
axes[1].set_ylabel("Proportion of individuals")
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend(bbox_to_anchor=(1.05, 1), loc="upper left", title="Origin")

# Grouped origin categories per habitat
bottom = np.zeros(len(indicator2_df_grouped))
bars_dict = {}
for i, cat in enumerate(grouped_cols):
    bars = axes[2].bar(
        indicator2_df_grouped["Habitat"],
        indicator2_df_grouped[cat],
        bottom=bottom,
        label=cat,
        color=grouped_colors[i]
    )
    bars_dict[cat] = (bars, bottom.copy())
    bottom += indicator2_df_grouped[cat].values

# Add percentage labels (>5%)
for cat, (bars, start) in bars_dict.items():
    for j, bar in enumerate(bars):
        height = bar.get_height()
        if height > 0.05:
            axes[2].text(
                bar.get_x() + bar.get_width() / 2,
                start[j] + height / 2,
                f"{height*100:.1f}%",
                ha="center", va="center", fontsize=9, color="white", fontweight="bold"
            )

axes[2].set_title("Grouped Species Origins per Habitat", fontsize=14, fontweight="bold")
axes[2].set_ylabel("Proportion of individuals")
axes[2].tick_params(axis='x', rotation=45)
axes[2].legend(loc="lower left", frameon=True)

plt.tight_layout()
plt.show()

NameError: name 'df_obs_sites' is not defined

In [ ]:
# Check if every observation successfully matched with an origin in df_species
unmatched_origin = df_obs_full[df_obs_full["Origin"].isna()]

print(f"Total unmatched observations (no origin found): {len(unmatched_origin)}")

if not unmatched_origin.empty:
    print("Unmatched species (no origin info found):")
    display(unmatched_origin[["clean_espece", "Transect", "Point", "Amount"]].drop_duplicates())
else:
    print("All observations successfully matched with a species origin.")

In [ ]:
# Indicator 3: Sampling Effort (Unique Transects Visited and Visit Intensity)

# Unique transects visited per year (+ mean line + values)
transects_per_year = (
    df_observations.groupby("year")["Transect"]
    .nunique()
    .reset_index(name="Unique_transects")
)

mean_unique = transects_per_year["Unique_transects"].mean()

plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=transects_per_year,
    x="year", y="Unique_transects",
    color="#4C72B0"
)

# Add values on top of bars
for i, val in enumerate(transects_per_year["Unique_transects"]):
    ax.text(i, val + 1, f"{val:.0f}", ha="center", va="bottom", fontsize=9, fontweight="bold")

# Add mean line
ax.axhline(mean_unique, color="red", linestyle="--", linewidth=2,
           label=f"Mean = {mean_unique:.1f}")

ax.set_title("Indicator 3 — Unique Transects Visited per Year", fontsize=14, fontweight="bold")
ax.set_ylabel("Number of unique transects visited")
ax.set_xlabel("Year")
ax.legend(loc="upper right")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Heatmap: visit intensity per transect and year (+ right panel: mean & median per transect)
visits_per_transect_year = (
    df_observations.groupby(["Transect", "year"])
    .size()
    .reset_index(name="Visits")
)

pivot_heatmap = visits_per_transect_year.pivot_table(
    index="Transect", columns="year", values="Visits", fill_value=0
)

mean_visits = pivot_heatmap.mean(axis=1)
median_visits = pivot_heatmap.median(axis=1)

pivot_heatmap_with_stats = pivot_heatmap.copy()
pivot_heatmap_with_stats["Mean"] = mean_visits
pivot_heatmap_with_stats["Median"] = median_visits

fig, (ax_hm, ax_stats) = plt.subplots(
    1, 2, figsize=(14, 8), gridspec_kw={"width_ratios": [8, 1]}, sharey=True
)

sns.heatmap(
    pivot_heatmap_with_stats.iloc[:, :-2],
    cmap="YlGnBu",
    annot=False,
    cbar_kws={"label": "Number of visits"},
    ax=ax_hm
)
ax_hm.set_title("Indicator 3 — Visit Intensity per Transect and Year", fontsize=14, fontweight="bold")
ax_hm.set_xlabel("Year")
ax_hm.set_ylabel("Transect")

ax_stats.barh(pivot_heatmap_with_stats.index, pivot_heatmap_with_stats["Mean"],
              color="#4C72B0", alpha=0.75, label="Mean")
ax_stats.barh(pivot_heatmap_with_stats.index, pivot_heatmap_with_stats["Median"],
              color="#55A868", alpha=0.75, label="Median")
ax_stats.set_title("Mean / Median", fontsize=11)
ax_stats.set_xlabel("Visits")
ax_stats.legend(loc="lower right", fontsize=9)

plt.tight_layout()
plt.show()